In [1]:
import numpy as np
import tensorflow as tf

2022-06-12 13:10:17.595046: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ajc348/.mujoco/mujoco210/bin:/home/ajc348/.mujoco/mujoco210/bin:/usr/local/Cluster-Apps/intel/2017.4/compilers_and_libraries_2017.4.196/linux/daal/lib/intel64_lin:/usr/local/Cluster-Apps/intel/2017.4/compilers_and_libraries_2017.4.196/linux/daal/../tbb/lib/intel64_lin/gcc4.4:/usr/local/Cluster-Apps/intel/2017.4/compilers_and_libraries_2017.4.196/linux/ipp/lib/intel64:/usr/local/Cluster-Apps/intel/2017.4/compilers_and_libraries_2017.4.196/linux/tbb/lib/intel64/gcc4.7:/usr/local/Cluster-Apps/intel/2017.4/debugger_2017/iga/lib:/usr/local/Cluster-Apps/intel/2017.4/debugger_2017/libipt/intel64/lib:/usr/local/Cluster-Apps/intel/2017.4/compilers_and_libraries_2017.4.196/linux/mpi/intel64/lib:/usr/local/Cluster-Apps/intel/2017.4/compilers_and_l

# Extract Rows

This would be applied before the data is split among batches.

In [2]:
data = np.reshape(np.arange(12, dtype=float), (4,3))
data

array([[ 0.,  1.,  2.],
       [ 3.,  4.,  5.],
       [ 6.,  7.,  8.],
       [ 9., 10., 11.]])

In [3]:
policies = np.array([0.,1.,0.,1.])[:, None]
policies

array([[0.],
       [1.],
       [0.],
       [1.]])

In [7]:
data[np.squeeze(np.argwhere(np.squeeze(policies)==0.)), :]

array([[0., 1., 2.],
       [6., 7., 8.]])

In [8]:
tf.gather(data, tf.squeeze(tf.where(tf.squeeze(policies)==0.)))

<tf.Tensor: shape=(2, 3), dtype=float64, numpy=
array([[0., 1., 2.],
       [6., 7., 8.]])>

In [9]:
tf.gather(data, tf.squeeze(tf.where(tf.squeeze(policies)==0.)))

<tf.Tensor: shape=(2, 3), dtype=float64, numpy=
array([[0., 1., 2.],
       [6., 7., 8.]])>

# Setup

In [6]:
rex_beta = 0.1

The below simulates the losses for an ensemble of 4 models, each passed 2 observerations with dimensionality of 3. This is what would have been produced in the original code.

In [327]:
losses_arr = np.reshape(np.arange(24, dtype=float), (4,3,2))
# losses_arr[0,0,:] = 0.
losses = tf.constant(losses_arr, dtype=float)
losses

<tf.Tensor: shape=(4, 3, 2), dtype=float32, numpy=
array([[[ 0.,  1.],
        [ 2.,  3.],
        [ 4.,  5.]],

       [[ 6.,  7.],
        [ 8.,  9.],
        [10., 11.]],

       [[12., 13.],
        [14., 15.],
        [16., 17.]],

       [[18., 19.],
        [20., 21.],
        [22., 23.]]], dtype=float32)>

Assigning policies to each record fed to each model. There are 3 policies, and so each model can never recieve an observation from every policy.

In [328]:
policies = tf.constant([
    [
        [1.],
        [1.],
        [2.],
    ],
    [
        [1.],
        [0.],
        [0.],
    ],
    [
        [0.],
        [2.],
        [2.],
    ],
    [
        [0.],
        [1.],
        [1.],
    ]

])
policies

<tf.Tensor: shape=(4, 3, 1), dtype=float32, numpy=
array([[[1.],
        [1.],
        [2.]],

       [[1.],
        [0.],
        [0.]],

       [[0.],
        [2.],
        [2.]],

       [[0.],
        [1.],
        [1.]]], dtype=float32)>

In [329]:
obs_losses = tf.reduce_mean(losses, axis=-1, keepdims=True)
obs_losses

<tf.Tensor: shape=(4, 3, 1), dtype=float32, numpy=
array([[[ 0.5],
        [ 2.5],
        [ 4.5]],

       [[ 6.5],
        [ 8.5],
        [10.5]],

       [[12.5],
        [14.5],
        [16.5]],

       [[18.5],
        [20.5],
        [22.5]]], dtype=float32)>

In [330]:
# unique_pols = tf.unique(tf.reshape(policies, [-1])).y
# unique_pols

In [331]:
# tf.where(policies==0.)[:,:2]

In [332]:
# tf.gather_nd(losses, tf.where(policies==0.)[:,:2], batch_dims=0)

In [333]:
# tf.gather_nd(losses, tf.cast(policies, tf.int32))[:,:,1,:]

# Solution

In [334]:
policies = tf.cast(policies, tf.int32)
policies

<tf.Tensor: shape=(4, 3, 1), dtype=int32, numpy=
array([[[1],
        [1],
        [2]],

       [[1],
        [0],
        [0]],

       [[0],
        [2],
        [2]],

       [[0],
        [1],
        [1]]], dtype=int32)>

In [335]:
unique_pols = tf.unique(tf.reshape(policies, [-1])).y
unique_pols

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 0], dtype=int32)>

In [336]:
# tf.reduce_max(unique_pols+1)

In [337]:
pol_one_hot = tf.squeeze(tf.one_hot(policies, tf.reduce_max(unique_pols+1), axis=-1), axis=-2)
pol_one_hot

<tf.Tensor: shape=(4, 3, 3), dtype=float32, numpy=
array([[[0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.]],

       [[0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]],

       [[1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.]],

       [[1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.]]], dtype=float32)>

In [338]:
pol_mean_sum = tf.squeeze(tf.matmul(tf.transpose(pol_one_hot, [0,2,1]), obs_losses), axis=-1)
pol_mean_sum

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 0. ,  3. ,  4.5],
       [19. ,  6.5,  0. ],
       [12.5,  0. , 31. ],
       [18.5, 43. ,  0. ]], dtype=float32)>

In [339]:
pol_count = tf.reduce_sum(pol_one_hot, axis=-2)
pol_count

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[0., 2., 1.],
       [2., 1., 0.],
       [1., 0., 2.],
       [1., 2., 0.]], dtype=float32)>

In [340]:
policy_losses = tf.math.divide_no_nan(pol_mean_sum, pol_count)
policy_losses

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 0. ,  1.5,  4.5],
       [ 9.5,  6.5,  0. ],
       [12.5,  0. , 15.5],
       [18.5, 21.5,  0. ]], dtype=float32)>

In [341]:
mean_policy_losses = tf.reduce_mean(policy_losses, axis=0)
mean_policy_losses

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([10.125,  7.375,  5.   ], dtype=float32)>

In [342]:
policy_total_losses = tf.reduce_sum(policy_losses, axis=-1)
policy_total_losses

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 6., 16., 28., 40.], dtype=float32)>

In [343]:
np.mean(np.arange(0.,6.)), np.mean(np.arange(6.,12.)), np.mean(np.arange(12.,18.)), np.mean(np.arange(18.,24.))

(2.5, 8.5, 14.5, 20.5)

In [344]:
# tf.math.reduce_sum(tf.ragged.boolean_mask(policy_losses, pol_count>0.),axis=-1)

In [345]:
# np.var(np.array((2.5))), np.var(np.array((10.,7.))), np.var(np.array((13.,16.))), np.var(np.array((19.,22.)))
# np.var(np.array((0.,4))), np.var(np.array((10.,7.))), np.var(np.array((13.,16.))), np.var(np.array((19.,22.)))
np.var(np.array((1.5,4.5))), np.var(np.array((9.5,6.5))), np.var(np.array((12.5,15.5))), np.var(np.array((18.5,21.5)))

(2.25, 2.25, 2.25, 2.25)

In [346]:
np.var(np.array((0.5,2.5,4.5)))

2.6666666666666665

In [347]:
tf.math.reduce_variance(tf.ragged.boolean_mask(policy_losses, pol_count>0.), axis=-1)

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([2.25, 2.25, 2.25, 2.25], dtype=float32)>

In [348]:
def determine_var(x):
    batch_pol_losses, batch_pol_counts = x[0,:], x[1,:]
    return tf.math.reduce_variance(tf.boolean_mask(batch_pol_losses, batch_pol_counts>0.))

In [349]:
policy_var_losses = tf.map_fn(determine_var, tf.stack((policy_losses, pol_count), axis=-2))
policy_var_losses

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([2.25, 2.25, 2.25, 2.25], dtype=float32)>

In [350]:
total_loss_var = policy_total_losses + policy_var_losses
total_loss_var

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 8.25, 18.25, 30.25, 42.25], dtype=float32)>

# Alternate

In [ ]:
tf.concat((losses, policies), axis=-1)

In [ ]:
def process_member(x):
    data, pol = x[:,:-1], x[:,-1:]
    def process_pol(y):
        mask = tf.tile(tf.equal(pol,y),(1,data.shape[-1]))
        return tf.reduce_mean(tf.boolean_mask(data, mask))
    return tf.map_fn(process_pol, unique_pols)

In [ ]:
result = tf.map_fn(process_member, tf.concat((losses, policies), axis=-1))

In [ ]:
result

In [ ]:
tf.reduce_sum(result, axis=-1)

In [ ]:
# tf.math.unsorted_segment_mean(losses, tf.cast(policies, tf.int64), 2)

# Original

In [ ]:
mean_pol = tf.where(tf.equal(tf.tile(policies,(1,1,losses.shape[-1])),unique_pols[0]), losses, tf.zeros_like(losses))
mean_pol

In [ ]:
tf.reduce_mean(mean_pol, axis=-1)

In [ ]:
tf.reduce_mean(tf.reduce_mean(mean_pol, axis=-1), axis=-1)

In [ ]:
mask = tf.tile(tf.equal(policies, unique_pols[0]), (1,1,3))
mask

In [ ]:
tf.boolean_mask(losses, mask)

In [ ]:
mask = tf.tile(tf.equal(policies[0,:,:], unique_pols[0]), (1,losses[0,:,:].shape[-1]))
mask

In [ ]:
tf.tile(tf.equal(policies[0,:,:], unique_pols[0]), (1,losses[0,:,:].shape[-1]))

In [ ]:
x = tf.concat((losses, policies), axis=-1)[0,:,:]
x

In [ ]:
data, pol = x[:,:-1], x[:,-1:]
data, pol

In [ ]:
mask = tf.tile(tf.equal(pol, unique_pols[0]), (1,data.shape[-1]))
tf.reduce_mean(tf.boolean_mask(data, mask))

In [ ]:
tf.concat((losses, policies), axis=-1)

In [ ]:
mask = tf.equal(policies, unique_pols[0])
tf.boolean_mask(losses, mask)

In [ ]:
tf.ragged.boolean_mask(losses, tf.tile(tf.equal(policies, unique_pols[1]),(1,1,3)))

In [ ]:
def simple_ret(x):
    return x[:,-1]

In [ ]:
tf.map_fn(simple_ret, tf.concat((mean, policies), axis=-1))

In [ ]:
logits = tf.constant([[[10.0, 10.0, 20.0, 20.0],
                      [11.0, 10.0, 10.0, 30.0],
                      [12.0, 10.0, 10.0, 20.0],
                      [13.0, 10.0, 10.0, 20.0]],
                     [[14.0, 11.0, 21.0, 31.0],
                      [15.0, 11.0, 11.0, 21.0],
                      [16.0, 11.0, 11.0, 21.0],
                      [17.0, 11.0, 11.0, 21.0]]])

indices = tf.constant([[[0, 0], [0, 1]], [[1, 1], [1, 3]]])

result = tf.gather_nd(logits, indices)
result

In [ ]:
logits.shape, indices.shape

In [ ]:
mean

In [ ]:
tf.slice(mean, tf.cast(policies, tf.int64))